In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_MN = pd.read_html('https://www.nass.usda.gov/Publications/AgCensus/2017/Online_Resources/Rankings_of_Market_Value/Minnesota/')



In [2]:
display(table_MN)

[                                                 Item  Farms Sales ($1,000)   
 0                                                 NaN    NaN            NaN  \
 1                                         Total sales  68822       18395390   
 2           Grains, oilseeds, dry beans, and dry peas  35101        8843440   
 3                                       Hogs and pigs   3562        3165075   
 4                                   Cattle and calves  18474        1886939   
 5                                      Milk from cows   3349        1737886   
 6                                                 NaN    NaN            NaN   
 7                                    Poultry and eggs   4175        1285951   
 8                                 Other crops and hay  18158         655825   
 9    Vegetables, melons, potatoes, and sweet potatoes   2860         383264   
 10         Nursery, greenhouse, floriculture, and sod    920         277254   
 11                                     

In [3]:
type(table_MN)

list

In [4]:
def map_columns(mapping=None, /, **kwargs):
    """
    Transform the specified columns and let the rest pass through.
    
    Examples:
    
        df.pipe(map_columns(a=lambda x: x + 1, b=str.upper))
        
        # dict for non-string column names
        df.pipe({(0, 0): np.sqrt, (0, 1): np.log10})
    """
    if mapping is not None and kwargs:
        raise ValueError("Only one of a dict and kwargs can be used at the same time")
    mapping = mapping or kwargs
    
    def map_columns_(df: pd.DataFrame) -> pd.DataFrame:
        mapping_funcs = {**{k: lambda x: x for k in df.columns}, **mapping}
        # preserve original order of columns
        return df.transform({key: mapping_funcs[key] for key in df.columns})
    return map_columns_
df =(table_MN[0]
     .drop(index=[0,1,6,11,16,19,20],axis=0)
     .drop(['Rank by Sales', 'Percent of Total Sales',],axis=1)
     .rename({"Sales ($1,000)":'Sales'},axis=1)
     .pipe(map_columns(Farms=pd.to_numeric))
     .pipe(map_columns(Sales=pd.to_numeric))
     .reset_index(drop=True)
)
print(df)
df.info()

                                                 Item  Farms    Sales
0           Grains, oilseeds, dry beans, and dry peas  35101  8843440
1                                       Hogs and pigs   3562  3165075
2                                   Cattle and calves  18474  1886939
3                                      Milk from cows   3349  1737886
4                                    Poultry and eggs   4175  1285951
5                                 Other crops and hay  18158   655825
6    Vegetables, melons, potatoes, and sweet potatoes   2860   383264
7          Nursery, greenhouse, floriculture, and sod    920   277254
8             Other animals and other animal products   1209    76204
9                Sheep, goats, wool, mohair, and milk   2673    26154
10                     Fruits, tree nuts, and berries   1118    25839
11         Horses, ponies, mules, burros, and donkeys   1414    14189
12                                        Aquaculture     60    11474
13  Cultivated Chris

In [5]:
pd.pivot_table(df,index="Item",values="Farms",aggfunc='sum',margins=True)

,Farms
Item,
Aquaculture,60
Cattle and calves,18474
Cultivated Christmas trees and short rotation woody crops,228
"Fruits, tree nuts, and berries",1118
"Grains, oilseeds, dry beans, and dry peas",35101
Hogs and pigs,3562
"Horses, ponies, mules, burros, and donkeys",1414
Milk from cows,3349
"Nursery, greenhouse, floriculture, and sod",920


In [7]:
pd.pivot_table(df,index="Item",values="Sales",aggfunc='sum',margins=True).sort_values(by=['Sales'], ascending=False)

,Sales
Item,
All,18395389
"Grains, oilseeds, dry beans, and dry peas",8843440
Hogs and pigs,3165075
Cattle and calves,1886939
Milk from cows,1737886
Poultry and eggs,1285951
Other crops and hay,655825
"Vegetables, melons, potatoes, and sweet potatoes",383264
"Nursery, greenhouse, floriculture, and sod",277254
